In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial.distance

In [17]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [18]:
train = pd.read_csv('./andan-2022-autumn-lab-3/train.csv')
#train.info()
#train['rubrics'].value_counts()
smooth = train.rating_org.mean()
means = train.groupby(['rubrics'])['rating_org'].mean()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73995 entries, 0 to 73994
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         73995 non-null  uint64 
 1   org_id                          73995 non-null  uint64 
 2   rating                          73995 non-null  float64
 3   ts                              73995 non-null  int64  
 4   user_city                       73995 non-null  object 
 5   org_city                        73995 non-null  object 
 6   average_bill                    45433 non-null  float64
 7   rating_org                      73995 non-null  float64
 8   rubrics                         73995 non-null  object 
 9   food_delivery                   73995 non-null  int64  
 10  breakfast                       73995 non-null  int64  
 11  takeaway                        73995 non-null  int64  
 12  summer_terrace                  

In [19]:
#not used
def split_in_right_categories(df):
    df['rubrics'] = df['rubrics'].replace(['Бар, паб'], 'Алкоголь')
    df['rubrics'] = df['rubrics'].replace(['Спортбар'], 'Алкоголь')
    df['rubrics'] = df['rubrics'].replace(['Кальян-бар'], 'Алкоголь')
    df['rubrics'] = df['rubrics'].replace(['Булочная, пекарня'], 'Булки')
    df['rubrics'] = df['rubrics'].replace(['Кондитерская'], 'Булки')
    df['rubrics'] = df['rubrics'].replace(['Пиццерия'], 'Этническое')
    df['rubrics'] = df['rubrics'].replace(['Суши-бар'], 'Этническое')
    df['rubrics'] = df['rubrics'].replace(['Бар безалкогольных напитков'], 'Горячие напитки')
    df['rubrics'] = df['rubrics'].replace(['Кофейня'], 'Горячие напитки')
    df['rubrics'] = df['rubrics'].replace(['Кофе с собой'], 'Горячие напитки')
    df['rubrics'] = df['rubrics'].replace(['Быстрое питание'], 'Дешево')
    df['rubrics'] = df['rubrics'].replace(['Столовая'], 'Дешево')
    return df

def split_in_right_params(df):
    df1 = pd.DataFrame()
    df1['HOME'] = [int(x1 == x2) for (x1, x2) in zip(df["user_city"], df["org_city"])]
    # df1['average_bill'] = df.average_bill
    df1['rating'] = (df.rating - df.rating.min())/(df.rating.max() - df.rating.min())
    df1['rubrics'] =  df.rubrics
    df1['TAKEAWAY'] = df.takeaway
    df1['DELIVERY'] = (df.food_delivery + df. food_product_delivery + df.delivery + df.free_delivery + df.online_takeaway)/6
    df1['LUST'] = (df.dancefloor + df.kalyan + df.strip)/3
    df1['DISABLED'] = (df.wheelchair_accessible + df.elevator_wheelchair_accessible + df.wheelchair_access + df.automatic_door + df.ramp + df.parking_disabled + df.toilet_for_disabled)/7
    df1['GAMES'] = (df.billiards + df.table_games)/2
    df1['CARDS'] = (df.accepted_credit_cards + df.payment_by_credit_card)/2
    df1['INTERNET'] = (df.wi_fi + df['internet access'])/2
    # df1['BUSINESS'] = df.breakfast + df.business_lunch
    df1['LUX'] = df.vip_zone + df.privilege_tickets + df.air_conditioning + df.call_button + df.vinotheque + df.car_park + df.summer_terrace
    df1['BEER'] = (df.craft_beer + df.has_bar)/2
    df1['MANY'] = (df.food_court1 + df.self_service_kiosks)/2
    df1['CHILL'] = (df.show_program + df.chillout + df.sports_broadcasts + df.karaoke + df.music)/5
    df1['HEALTHY'] = (df.organic_food + df.farm_products + df.handmade_goods)/3
    df1['FAST'] = (df.coffee_to_go + df.minimum_order)/2
    df = df1
    return df
#train = split_in_right_categories(train)
#train['rubrics'].value_counts()

In [20]:
'''
def get_rubrics(df):
    rubrics = df.rubrics.unique()
    rubrics_dict = {}

    for rubric in rubrics:
        all_values_of_rubric = df[df["rubrics"] == rubric].loc[:, ["average_bill"]].dropna()
        rubrics_dict[rubric] = all_values_of_rubric.average_bill.median()

    return rubrics_dict

def fill_na_bill(df):
    rubrics_dict = get_rubrics(df)
    value = df.rubrics
    value = value.apply(lambda x: rubrics_dict[x])

    df.average_bill.fillna(inplace=True, value=value)
    #return df
'''

'\ndef get_rubrics(df):\n    rubrics = df.rubrics.unique()\n    rubrics_dict = {}\n\n    for rubric in rubrics:\n        all_values_of_rubric = df[df["rubrics"] == rubric].loc[:, ["average_bill"]].dropna()\n        rubrics_dict[rubric] = all_values_of_rubric.average_bill.median()\n\n    return rubrics_dict\n\ndef fill_na_bill(df):\n    rubrics_dict = get_rubrics(df)\n    value = df.rubrics\n    value = value.apply(lambda x: rubrics_dict[x])\n\n    df.average_bill.fillna(inplace=True, value=value)\n    #return df\n'

In [21]:
def calculate_rubrics(df):
    rubric_num = {}
    unique = df.rubrics.unique()

    for i in range(0, len(unique)):
        rubric_num[unique[i]] = i

    df.rubrics = df.rubrics.apply(lambda x: rubric_num[x])
    
    

In [22]:
#fill_na_bill(train)
#split_test_by = int(0.8*len(train))
#train = train.iloc[:split_test_by, :]
#calculate_rubrics(train)
#train_rating = train.rating_org
#print(train.isnull().values.any())
#train = split_in_right_params(train)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error

def optimal_neighbour(df):
    split_test_by = int(0.8*len(df))
    tpredict = df['rating_org'][:split_test_by]
    ttestpredict = df['rating_org'][split_test_by:]    
    
    # fill_na_bill(df)
    calculate_rubrics(df)
    df = split_in_right_params(df)
    ttrain = df.iloc[:split_test_by, :]
    ttest = df.iloc[split_test_by:, :]
    if tpredict.isnull().values.any() or ttrain.isnull().values.any() or ttestpredict.isnull().values.any() or ttest.isnull().values.any():
        print(True)
        return -1
    else:
        print(False)
    def neighbour_predict(neighbours):
        model = KNeighborsRegressor(n_neighbors=neighbours, n_jobs=8)
        model.fit(ttrain, tpredict)
        
        predicted = model.predict(ttest)
        mape = mean_absolute_percentage_error(ttestpredict , predicted)
        return mape
    i = 1
    min = -1
    mape1 = 1000000
    while i <= 22:
        mape2 = neighbour_predict(i)
        if(mape1 > mape2):
            mape1 = mape2
            min = i
        i += 1
    return min
tmp_train = train
on = optimal_neighbour(tmp_train)
print(on)
del tmp_train

In [23]:
#fill_na_bill(train)
calculate_rubrics(train)
train_rating = train.rating_org
train = split_in_right_params(train)

train.corrwith(train_rating)

HOME       -0.057218
rating      0.269386
rubrics    -0.040596
TAKEAWAY    0.058840
DELIVERY   -0.029985
LUST       -0.035468
DISABLED    0.186717
GAMES       0.030478
CARDS       0.151706
INTERNET    0.081437
LUX         0.068824
BEER        0.122229
MANY       -0.010682
CHILL       0.068830
HEALTHY     0.072902
FAST       -0.054986
dtype: float64

In [24]:
neigh = KNeighborsRegressor(n_neighbors=on, n_jobs=8)
neigh.fit(train, train_rating)

KNeighborsRegressor(n_jobs=8, n_neighbors=14)

In [25]:
neigh.score(train, train_rating)

0.35155773378006205

In [26]:
test = pd.read_csv('./andan-2022-autumn-lab-3/test_x.csv')
test['avg_ratings'] = 0.0
for index, row in test.iterrows():
    test.at[index,'avg_ratings'] = means[row['rubrics']]
#test = split_in_right_categories(test)
#fill_na_bill(test)
calculate_rubrics(test)
avg_tests = test['avg_ratings']

In [27]:
test = split_in_right_params(test)

In [28]:
#print(avg_tests)

0        4.351358
1        4.351358
2        4.317091
3        4.317091
4        4.317091
           ...   
24660    4.351358
24661    4.317091
24662    4.376219
24663    4.376219
24664    4.351358
Name: avg_ratings, Length: 24665, dtype: float64


In [ ]:
result = neigh.predict(test)
#result = (25 * avg_tests + 25 * smooth + 950 * neigh.predict(test))/1000
pd.DataFrame(result).to_csv('./andan-2022-autumn-lab-3/my_sub.csv')
# result